In [6]:
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.regularizers import l2, activity_l2,l1
from keras.utils import np_utils
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.vgg16 import preprocess_input
from scipy import misc
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
import scipy

Using Theano backend.
Using gpu device 1: Tesla K20c (CNMeM is disabled, cuDNN 4007)


### Generate model with VGG16 feature extractors, set trainable false

In [20]:
def getModel( output_dim ):
    # output_dim: the number of classes (int)
    # return: compiled model (keras.engine.training.Model)
    
    vgg_model = VGG16( weights='imagenet', include_top=True )
    vgg_out = vgg_model.layers[-2].output 
    
    vgg_out = Dropout(0.25)(vgg_out)
    softmax = Dense( output_dim, activation="softmax", W_regularizer = l2(0.05))( vgg_out )
    

    tl_model = Model( input=vgg_model.input, output=softmax )
    # Transfer Learning
    for layer in tl_model.layers[0:-1]:
        layer.trainable = False            

    tl_model.compile(loss= "categorical_crossentropy", optimizer="adadelta", metrics=["acc"])
    
    return tl_model

### Loading all images into 'album'

In [8]:
def loadImages(path = '/mnt/cube/UT_/Urban_tribes/'):
    album = {}
    for item in listdir(path):
        category = item[0:4]
        if category == ".ipy":
            continue
        if category not in album:
            album[category] = []
        
        img = load_img(path+item)
        img = img_to_array(img)
        img = misc.imresize(img, (224, 224))
        img = scipy.misc.imrotate(img, 180)
        album[category].append(img)
    return album

In [9]:
album = loadImages()

In [10]:
# plt.imshow(album['goth'][0][:, :, 0])
# plt.show()

### Split 'album' into dataset
Album is split into training and testing input/outputs according to number of examples attribute

In [11]:
def make_dataset(album, n_train, n_test):
    trn_inp = []
    trn_out = []
    tst_inp = []
    tst_out = []    
    keys = album.keys()
    for key in keys:
        examples = album[key]
        l = len(examples)
        idx = np.random.choice(l, n_train+n_test)
        for i in idx[:-n_test]:
            trn_inp.append(examples[i])
            trn_out.append(keys.index(key))
        for i in idx[-n_test:]:
            tst_inp.append(examples[i])
            tst_out.append(keys.index(key))
    return [trn_inp, trn_out, tst_inp, tst_out]      

In [12]:
# plt.imshow(album['bike'][0][:, :, 0])
# # plt.show()
# album.keys()

In [26]:
samCat = 8
dataset = make_dataset(album, samCat, 8)

In [27]:
len(dataset[1])

88

In [28]:
trainX = np.array(dataset[0])
trainY = np.array(dataset[1])

# Now shuffle the training data and swapaxes
idx = np.random.choice(len(trainX), len(trainX))
trainX = trainX[idx]
trainY = trainY[idx]

trainX = preprocess_input(np.float64(trainX)).swapaxes(1, 3).swapaxes(2, 3)
trainY = np_utils.to_categorical(trainY)

# Only swapaxes for testing data
testX = np.array(dataset[2])
testY = np.array(dataset[3])

testX = preprocess_input(np.float64(testX)).swapaxes(1, 3).swapaxes(2, 3)
testY = np_utils.to_categorical(testY)

In [29]:
# plt.imshow(trainX[10][0, :, :])
# plt.show()

### Fit the model

In [30]:
model = getModel(11)

In [31]:
history = model.fit(trainX, trainY, batch_size = 8, nb_epoch = 30, validation_data = (testX, testY), shuffle=True)

Train on 88 samples, validate on 88 samples
Epoch 1/30
88/88 [==============================] - 4s - loss: 4.2543 - acc: 0.1364 - val_loss: 3.8209 - val_acc: 0.1818
Epoch 2/30
88/88 [==============================] - 4s - loss: 2.8818 - acc: 0.3636 - val_loss: 3.8739 - val_acc: 0.2273
Epoch 3/30
88/88 [==============================] - 4s - loss: 2.1218 - acc: 0.6364 - val_loss: 3.8493 - val_acc: 0.2727
Epoch 4/30
88/88 [==============================] - 4s - loss: 1.7329 - acc: 0.6705 - val_loss: 3.7605 - val_acc: 0.2841
Epoch 5/30
88/88 [==============================] - 4s - loss: 1.4922 - acc: 0.8295 - val_loss: 3.7423 - val_acc: 0.2727
Epoch 6/30
88/88 [==============================] - 4s - loss: 1.3431 - acc: 0.8750 - val_loss: 3.7160 - val_acc: 0.2727
Epoch 7/30
88/88 [==============================] - 4s - loss: 1.2270 - acc: 0.9205 - val_loss: 3.6295 - val_acc: 0.2727
Epoch 8/30
88/88 [==============================] - 4s - loss: 1.1306 - acc: 0.9545 - val_loss: 3.6120 - val_

In [25]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss for %d samples per category' % samCat)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

plt.plot(history.history['val_acc'])
plt.title('model accuracy for %d samples per category' % samCat)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
val_acc: 0.2273 2 sample
        0.25  4 sample